In [ ]:
import os
import csv
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
#!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
import json

TOKEN = {"username":"tianweili","key":"1911ddc99462ba042344b26539482486"}

! pip install kaggle==1.5.12
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle/

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content

     |████████████████████████████████| 58 kB 5.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=fdd6dcd220e1a41fb7697ef27b990281dff5417d64a338d1c808ea9be5c82f3e
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
kaggle.json
- path is now set to: /content


In [ ]:
!rm -r hw1p2_student_data

rm: cannot remove 'hw1p2_student_data': No such file or directory


In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!kaggle competitions download -c 11-785-s22-hw1p2

100% 1.85G/1.86G [00:14<00:00, 154MB/s]
100% 1.86G/1.86G [00:14<00:00, 137MB/s]


In [ ]:
! unzip competitions/11-785-s22-hw1p2/11-785-s22-hw1p2.zip

Streaming output truncated to the last 5000 lines.
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-020.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-021.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-022.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-023.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-024.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-025.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-026.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-027.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-028.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-029.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-030.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript

In [ ]:
! kaggle competitions submit -c 11-785-s22-hw1p2 -f submission.csv -m "A test submission"

100% 18.6M/18.6M [00:03<00:00, 6.46MB/s]
Successfully submitted to Frame-Level Speech Recognition

In [ ]:
!ls

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # TODO: Please try different architectures
        in_size = 845
        layers = [
            nn.Linear(in_size, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.BatchNorm1d(256),
            nn.Linear(256, 40)
        ]
        self.laysers = nn.Sequential(*layers)
    
    def init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            m.bias.data.fill_(0.01)

    def forward(self, A0):
        x = self.laysers(A0)
        return x

In [ ]:
class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition +"/transcript/"
        
        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]
        
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
        
        assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]
            
            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))
            
        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y
    
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context = 0):
        assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        self.context = context

        if context == 0:
            self.X, self.Y = X, Y
        else:
            # TODO: self.X, self.Y = ... 
            self.X = X
            self.Y = Y
            self.context = context

            self.X = np.pad(self.X, ((context, context), (0, 0)), 
                                        "constant", 
                                        constant_values=0)          
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            # TODO xx, yy = ...
            xx = self.X[i : i + 2 * self.context + 1].flatten()
            # print(xx)
            yy = self.Y[i]
        return xx, yy
    



In [ ]:
from numpy.core.fromnumeric import partition
def train(args, model, device, train_samples, optimizer, criterion, epoch, scaler, num_workers):
    model.train()
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True, num_workers=num_workers)

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(data)
                loss = criterion(output, target)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            #optimizer.step()
            scaler.update()
            if batch_idx % args['log_interval'] == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, dev_samples, num_workers):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False, num_workers=num_workers)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())

    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    return train_accuracy


def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = Network().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    criterion = torch.nn.CrossEntropyLoss()
    num_workers = 2

    scaler = torch.cuda.amp.GradScaler()
    # If you want to use full Dataset, please pass None to csvpath

    #, csvpath="/content/drive/MyDrive/train_filenames_subset_0512_v2.csv")
    
    train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100")
    dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")

    for epoch in range(1, args['epoch'] + 1):
        train(args, model, device, train_samples, optimizer, criterion, epoch, scaler, 2)
        
        if epoch % 5 == 0:
            print('train accuracy: ', test(args, model, device, train_samples, 2))
        
        test_acc = test(args, model, device, dev_samples, 2)
        print('Dev accuracy ', test_acc)
    
    
    test_samples = TestSamples(data_path = args['LIBRI_PATH'], partition='test-clean', csvpath='/content/drive/MyDrive/test_order.csv')
    df = pred(args, model, device, test_samples)
    
    
    

if __name__ == '__main__':
    args = {
        'batch_size': 2048,
        'context': 32,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 0.001,
        'epoch': 13
    }
    main(args)

Train Epoch: 1 [0/25337037 (0%)]	Loss: 3.857986
Train Epoch: 1 [409600/25337037 (2%)]	Loss: 1.248420
Train Epoch: 1 [819200/25337037 (3%)]	Loss: 1.050243
Train Epoch: 1 [1228800/25337037 (5%)]	Loss: 0.956639
Train Epoch: 1 [1638400/25337037 (6%)]	Loss: 0.958579
Train Epoch: 1 [2048000/25337037 (8%)]	Loss: 0.930953
Train Epoch: 1 [2457600/25337037 (10%)]	Loss: 0.925158
Train Epoch: 1 [2867200/25337037 (11%)]	Loss: 0.871822
Train Epoch: 1 [3276800/25337037 (13%)]	Loss: 0.843075
Train Epoch: 1 [3686400/25337037 (15%)]	Loss: 0.846831
Train Epoch: 1 [4096000/25337037 (16%)]	Loss: 0.827626
Train Epoch: 1 [4505600/25337037 (18%)]	Loss: 0.819415
Train Epoch: 1 [4915200/25337037 (19%)]	Loss: 0.858542
Train Epoch: 1 [5324800/25337037 (21%)]	Loss: 0.768854
Train Epoch: 1 [5734400/25337037 (23%)]	Loss: 0.756998
Train Epoch: 1 [6144000/25337037 (24%)]	Loss: 0.725407
Train Epoch: 1 [6553600/25337037 (26%)]	Loss: 0.767440
Train Epoch: 1 [6963200/25337037 (27%)]	Loss: 0.755161
Train Epoch: 1 [7372800/

In [ ]:
class TestSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, partition="test-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        
        self.X_names = os.listdir(self.X_dir)

        # using a small part of the dataset to debug
        if csvpath:
            self.X_names = list(pd.read_csv(csvpath).file)

        self.length = len(self.X_names)

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X = []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            
        X = np.concatenate(X)
        return X
    
class TestItems(torch.utils.data.Dataset):
    def __init__(self, X, context=0):
        self.length  = X.shape[0]

        if context == 0:
            self.X = X
        else:
            # TODO: self.X, self.Y = ... 
            self.X = X
            self.context = context

            self.X = np.pad(self.X, ((context, context), (0, 0)), 
                                        "constant", 
                                        constant_values=0)
              
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
        else:
            # TODO xx, yy = ...
            xx = self.X[i : i + 2 * self.context + 1].flatten()
        return xx

def pred(args, model, device, test_samples):
    model.eval()
    
    pred_y_list = []
    #df = pd.DataFrame(columns=['id', 'label'])
    with torch.no_grad():
        count = 0
        for i in range(len(test_samples)):
            X = test_samples[i]

            test_items = TestItems(X, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data in test_loader:
                data = data.float().to(device)               
                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
                if count < 200:
                   count += 1
                
                if count == 200:
                    print(pred_y_list)
                    count = 201

        df = pd.DataFrame(pred_y_list, columns=['label'])
        #df['id'] = df.index
        #df = df[['id', 'label']]
        df.index.name = 'id'
        print(df.head())
        print(len(df))
        print(df.tail())
        df.to_csv('/content/submission.csv')

    return df

In [ ]:
! ls /content/drive/MyDrive/

'Agile Assignment 3 Team 7.gdoc'	 test_order.csv
'Cohort Assignment 1.gdoc'		"Tianwei Li's Resume.pdf"
'Colab Notebooks'			 train_filenames_subset_0008_v2.csv
 new中秋歌唱比赛节目表.gsheet		 train_filenames_subset_8192_v2.csv
'ODI Final Paper.gdoc'			'Untitled document (1).gdoc'
'ODI Leadership Reflection.gdoc'	'Untitled document (2).gdoc'
'Reflection of Five Dysfuncitons.gdoc'	'Untitled document (3).gdoc'
 submission.csv				'Untitled document.gdoc'
